# Amazonia-1 on AWS Open Data

## Accessing Cloud Optimized Multi-Band Raster Data


With the AWS ASDI STAC API we can search and load various geospatial datasets. Below we'll work with the Amazonia-1 Satelite WFI Camera, Level-4 Orthorectified data, sourced from Cloud Optimized GeoTIFFs (COGs) [hosted on AWS](https://registry.opendata.aws/amazonia/). In this example, we read in 4 band imagery within a bounding box.

First we install any necessary packages. Please note that the following block of code only needs to be run if needed within a new workspace and that you may need to restart the kernel to use updated packages. The code block only needs to be run once for each AWS Sagemaker Studio profile.

In [8]:
%pip install --upgrade pip -q
%pip install matplotlib 'pystac_client>=0.5.0' stackstac rasterio folium httpx -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# All imports in the order they are used in the code
import os
from pystac_client import Client
import stackstac

import shapely as shp
import folium

import rasterio

We'll use `pystac_client` to submit our search query, stackstac to load and composite our rasters as an Xarray DataArray, and rio-tiler to performantally load individual COGs.

### Querying the AWS STAC API with Pystac Client

First, we define the URL to our STAC API, a service that allows us to submit search queries for spatio-temporal assets (like elevation raster tiles).

Next, we open this URL with `pystac_client`, which allows us to use the client's search method. `client.search` accepts many different arguments to specify the search query. Common query parameters are `time_range` and `bbox` (the spatial bounding box). Since the DEM was collected at one point in time, we will only specify the "where" with a `bbox` describing the minx, miny, maxx, maxy coordinates in lat/lon degrees.

In [1]:
import os
from pystac_client import Client
import stackstac

# making a connection to AWS STAC API
API_ROOT_URL = "https://dev.asdi-catalog.org"
client = Client.open(API_ROOT_URL)

# Search near Curitiba Brazil
bbox = [-49.320232, -25.626009, -48.927635, -25.323504]
source_search = client.search(
        collections=["AMAZONIA1-WFI"], 
        bbox=bbox,
        max_items=100
    )
# getting the metadata and links to data assets in STAC format
source_items = source_search.item_collection()
print("Number of WFI tiles: ", len(source_items))

Number of WFI tiles:  15


In [6]:
# Look at the search parameters
search_request = source_search.get_parameters()
search_request["filter-lang"] = "cql_json"
search_request

{'limit': 100,
 'bbox': (-49.320232, -25.626009, -48.927635, -25.323504),
 'collections': ('AMAZONIA1-WFI',),
 'filter-lang': 'cql_json'}

In [12]:
import httpx

endpoint = "https://titiler.dev.asdi-catalog.org"
print(httpx.get(f"{endpoint}/healthz").json())

{'database_online': True}


In [ ]:
response = httpx.post(
    f"{endpoint}/mosaic/register", json=search_request,
).json()
print(response)

searchid = response["searchid"]

### Loading an OpenStreetMap(OSM) descriptive map of the same area

To better understand our area of interest, we can create a descriptive map with our bounding box. We'll load an OpenStreetMap basemap that includes place names, annotations for water and forested areas, and roads. We'll use `folium` to access and plot this map. To learn more about `folium`, check out the [documentation](https://python-visualization.github.io/folium/). We'll use our bounding box to specify where we want to center our map and pick a zoom level that shows us detail around San Ignacio.

In [4]:
# import shapely for manipulation and analysis of geometric objects
# in this case to find the center of the bounding box bbox
import shapely as shp
bbox_poly = shp.geometry.box(*bbox)
center_point = (bbox_poly.centroid)

In [5]:
import folium
m = folium.Map(location=[center_point.y, center_point.x], zoom_start=6)
bbox_style_blue = {'fillColor': '#3388ff', 'color': '#3388ff', 'fillOpacity':'0.05'} #make bbox red
folium.GeoJson(source_items.to_dict(), name="items", style_function=lambda x:bbox_style_blue).add_to(m)
bbox_style_red = {'fillColor': '#ff0000', 'color': '#ff0000'} #make bbox red
folium.GeoJson(bbox_poly, name="bbox", style_function=lambda x:bbox_style_red).add_to(m)
m

### Explore the Items

The query resulted in 15 scenes that match the query. Lets look more closely at the properties in the STAC records.

To Start you can print a single Item, and Jupyter will give you an interactive tree to explore.

Since all the data is from the same bounding box, this is also a timeseries of 15 different times for the same location

## Explore Scenes

Let plot a single band for each time, reading only the data we need right when it's needed. 

> If this had a been a bigger selection of data falling back on thumbnail Assets would be more efficient.

Turns out clouds are a big issue, so need to find a time without clouds. With the following code block you will see only 5 of the 15 times are good data.

In [ ]:
tj_response = httpx.get(f"{endpoint}/mosaic/{searchid}/tilejson.json?assets=cog").json()
print(tj_response)

In [ ]:
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=14
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)

aod_layer = TileLayer(
    tiles=tj_response["tiles"][0],
    attr="Mosaic",
    min_zoom=14,
    max_zoom=18,
    max_native_zoom=18,
)
aod_layer.add_to(m)
m

#### Plotting 3 Bands

Now that we've found good data, we can plot a 3 band image, matching the Bands to RGB for display.

For the Amazonia WFI sensor if you look at the Asset properties on the STAC item you can see the bands labelled.


In [12]:
print(stack.band.values)
print(stack.common_name.values)

['B1' 'B2' 'B3' 'B4']
['blue' 'green' 'red' 'nir']


Now for the first example in true color Red, Green, Blue.


Then for the second in false color infrared, Near Infrared, Red, Green.